In [2]:
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI


In [3]:
load_dotenv()
API_KEY=os.getenv("KEY")

In [4]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(openai_api_key=API_KEY,temperature=0.5,model ="gpt-3.5",max_retries=3)

In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
TEMPLATE="""Text: {text}
You are an expert at creating multiple choice questions for quizzes.Give the above text and generate {number} multiple choice questions on the subject of {subject} in a {tone} tone.
Make sure the questions are diverse and cover different aspects of the text and are not repetitive.make sure to format the response in this json format {response_json}
make sure to make {number} questions only.
#Response JSON Format:
{response_json}"""

In [8]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)


In [9]:
quiz_chain=LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="quiz",
    verbose=True
)


C:\Users\hp\AppData\Local\Temp\ipykernel_14268\540580881.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(


In [10]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [11]:
quiz_evalaution_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE2
)

In [12]:
review_chain=LLMChain(
    llm=llm,
    prompt=quiz_evalaution_prompt,
    output_key="reviewed_quiz",
    verbose=True
)

In [13]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","reviewed_quiz"],
    verbose=True
)

In [14]:
file_path="D:\mcqgen\experiment\data.txt"

In [15]:
with open (file_path, 'r', encoding='utf-8') as file:
    TEXT = file.read()

In [16]:
json.dumps(RESPONSE_JSON, indent=4)

'{\n    "1": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "2": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    },\n    "3": {\n        "mcq": "multiple choice question",\n        "options": {\n            "a": "choice here",\n            "b": "choice here",\n            "c": "choice here",\n            "d": "choice here"\n        },\n        "correct": "correct answer"\n    }\n}'

In [17]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {   
            "text": TEXT,
            "number": 5,
            "subject": "PHYSICS",
            "tone": "tone",
            "response_json": json.dumps(RESPONSE_JSON, indent=4)
        }
    )

C:\Users\hp\AppData\Local\Temp\ipykernel_14268\77120582.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
Text: Physics is the scientific study of matter, its fundamental constituents, its motion and behavior through space and time, and the related entities of energy and force.[1] It is one of the most fundamental scientific disciplines.[2][3][4] A scientist who specializes in the field of physics is called a physicist.

Physics is one of the oldest academic disciplines.[5] Over much of the past two millennia, physics, chemistry, biology, and certain branches of mathematics were a part of natural philosophy, but during the Scientific Revolution in the 17th century, these natural sciences branched into separate research endeavors. Physics intersects with many interdisciplinary areas of research, such as biophysics and quantum chemistry, and the boundaries of physics are not rigidly defined. New ideas in physics often explain the fundamental mechanisms studied by other sciences[2] and suggest new avenues of research in these and other academic disciplines such as mat

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-3.5` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [18]:
quiz=response.get("quiz")
quiz=json.loads(quiz)

NameError: name 'response' is not defined

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value['mcq']
    options = "l".join([
        f"{option}:{option_value}"
        for option, option_value in value['options'].items()
    ])
    correct = value['correct']
    quiz_table_data.append({
        "MCQ": mcq,
        "Options": options,
        "Correct ": correct
    })

In [ ]:
quiz_table_data
pd.DataFrame(quiz_table_data)
quiz.to_csv("Physics_quiz.csv", index=False)